In [1]:
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 

np.random.seed(42)

In [2]:
# build own load data function
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

    # normalize
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))

    x_train = (x_train - mean) / (std+1e-7)
    x_test = (x_test - mean) / (std+1e-7)

    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test

In [9]:
# build cifar-10 deep cnn model
def build_model(input_shape=(32, 32, 3), classes=10):
    model = models.Sequential()

    # 1st block
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))

    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))

    # top layers
    model.add(layers.Flatten())
    model.add(layers.Dense(classes, activation='softmax'))

    return model

In [10]:
(x_train, y_train, x_test, y_test) = load_data()

model = build_model()
model.summary()
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.TensorBoard('./logs')
]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)       

In [11]:
# image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

datagen.fit(x_train)

In [12]:
# train model
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs = 100,
    verbose = 1,
    validation_data = (x_test, y_test))

# save to disk
model_json = model.to_json()

with open('weights/model_cifar.json', 'w') as file:
    file.write(model_json)

model.save_weights('weights/model_cifar.h5')


# test 
score = model.evaluate(x_test, y_test,
    verbose=1,
    batch_size=64)

print('Test result : {:1.5f} loss: {:1.5f}'.format(scores[1] * 100, scores[0]))

  ...
    to  
  ['...']
Train for 782 steps, validate on 10000 samples
Epoch 1/100
  1/782 [..............................] - ETA: 1:17:31

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/conv2d_6/Conv2D (defined at <ipython-input-12-07b841dc9815>:6) ]] [Op:__inference_distributed_function_5493]

Function call stack:
distributed_function
